<a href="https://colab.research.google.com/github/Servat0r/HLT-Project-2023/blob/master/LMQG_Squad_10000_examples_Modified_Input.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks"

/content/drive/MyDrive/Colab Notebooks


In [ ]:
%run utils.ipynb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.6 MB/s eta 0:00:0

In [ ]:
NUM_EPOCHS = 10

#### Imports

In [ ]:
from transformers import AutoModel, AutoTokenizer, MT5ForConditionalGeneration, TrainingArguments, Trainer, AdamW, DataCollatorWithPadding
from datasets import load_dataset, Dataset, load_from_disk, load_metric
import numpy as np
import evaluate
import torch
import os

In [ ]:
model_checkpoint='google/mt5-base'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
_POSSIBLE_CONJ = {
    'who': 'Who', 'in what': 'In what', 'at what': 'At what', 'what': 'What',
    'when': 'When', 'why': 'Why', 'how many': 'How many', 'how': 'How',
    'where': 'Where', 'in which': 'In which', 'which': 'Which'
}

def build_train_feature_lmqg_squad_highlighting(sample, use_extra_ids=False):
    question_text = sample['question'].lower()
    start = ''
    for conj in _POSSIBLE_CONJ:
        if conj in question_text:
            start = f" that start with {_POSSIBLE_CONJ[conj]}"
            break
    return {'answer_context': f"generate questions{start}: <answer> {sample['answer']} <answer> <context> {sample['paragraph_answer']} <context>"}

In [ ]:
def load_and_preprocess_lmqg_squad_dataset_highlighting(
    dataset_name='lmqg/qg_squad', train_dataset_name='lmqg_qg_squad_highlighting_train',
    eval_dataset_name='lmqg_qg_squad_highlighting_eval', test_dataset_name='lmqg_qg_squad_highlighting_test',
    shuffle_seed=None, train_select=None, eval_select=None, use_extra_ids=False,
):
  dataset_loading_result = get_dataset(dataset_name, train_dataset_name, eval_dataset_name, test_dataset_name)
  local = dataset_loading_result['local']
  if local:
    train_dataset = dataset_loading_result['train']
    validation_dataset = dataset_loading_result['eval']
    test_dataset = dataset_loading_result['test']
  if not local:
    datasets = load_dataset(dataset_name)
    print(datasets['test'])
    train_dataset = datasets['train'].remove_columns(['paragraph_question', 'sentence', 'sentence_answer', 'paragraph', 'paragraph_sentence'])
    validation_dataset = datasets['validation'].remove_columns(['paragraph_question', 'sentence', 'sentence_answer', 'paragraph', 'paragraph_sentence'])
    test_dataset = datasets['test'].remove_columns(['paragraph_question', 'sentence', 'sentence_answer', 'paragraph', 'paragraph_sentence'])
    print(f"Train dataset has {len(train_dataset)} items. Validation dataset has {len(validation_dataset)} items.")

    train_dataset.save_to_disk(train_dataset_name)
    validation_dataset.save_to_disk(eval_dataset_name)
    test_dataset.save_to_disk(test_dataset_name)

  if train_select:
    train_dataset = train_dataset.shuffle(seed=0).select(range(train_select))
  if eval_select:
    validation_dataset = validation_dataset.shuffle(seed=0).select(range(eval_select))

  print(train_dataset)

  build_train_feature = lambda sample: build_train_feature_lmqg_squad_highlighting(sample, use_extra_ids=use_extra_ids)
  train_dataset = train_dataset.map(build_train_feature).remove_columns(['answer', 'paragraph_answer'])
  validation_dataset = validation_dataset.map(build_train_feature).remove_columns(['answer', 'paragraph_answer'])
  test_dataset = test_dataset.map(build_train_feature).remove_columns(['answer', 'paragraph_answer'])

  tokenizer_function_lambda = lambda sample: tokenizer_function(sample, train_dataset=train_dataset)
  tokenized_train_dataset = train_dataset.map(tokenizer_function_lambda, batched=True).remove_columns(['answer_context', 'question'])
  tokenized_validation_dataset = validation_dataset.map(tokenizer_function_lambda, batched=True).remove_columns(['answer_context', 'question'])
  tokenized_test_dataset = test_dataset.map(lambda samples: tokenizer_function(samples, input_ids_padding="max_length"), batched=True).remove_columns(['answer_context', 'question'])

  tokenized_train_dataset.set_format("torch")
  tokenized_validation_dataset.set_format("torch")
  tokenized_test_dataset.set_format("torch")

  return (train_dataset, validation_dataset, test_dataset), (tokenized_train_dataset, tokenized_validation_dataset, tokenized_test_dataset)

### 2. Dataset Loading and Preprocessing

We will use [`squad_it`](https://huggingface.co/datasets/squad_it), a variant of the `SquaD` dataset adapted to `Question Generation` tasks.

In [ ]:
(train_dataset, validation_dataset, test_dataset), (tokenized_train_dataset, tokenized_validation_dataset, tokenized_test_dataset) = \
  load_and_preprocess_lmqg_squad_dataset_highlighting(shuffle_seed=42, train_select=10000, eval_select=5000, use_extra_ids=True)

Dataset({
    features: ['answer', 'question', 'paragraph_answer'],
    num_rows: 10000
})


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/11877 [00:00<?, ? examples/s]

### 3. Model Loading and configuration

#### Loading

In [ ]:
model = MT5ForConditionalGeneration.from_pretrained(model_checkpoint)

#### Configuration

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [ ]:
optimizer, train_dataloader, eval_dataloader, lr_scheduler, \
  num_training_steps = get_training_configuration(train_batch_size=4, eval_batch_size=4, tokenizer=tokenizer, learning_rate=1e-4, num_epochs=NUM_EPOCHS)

25000


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


### 4. Fine-tuning

#### Execution

In [ ]:
training_results_dict = main_training_loop(
    model, device, optimizer, train_dataloader, eval_dataloader,
    lr_scheduler, num_training_steps, num_epochs=NUM_EPOCHS, metrics=None,
    eval_strategy='epoch', eval_every=2000, model_save_path='lmqg_squad_reduced_mt5base_conj_10000examples_test',
    early_stopping=False, early_stopping_patience=4, tokenizer=tokenizer,
    num_beams=4, tokenize_predictions_output=False, start_epoch=0
  )

epoch_train_losses = training_results_dict['epoch_train_losses']
epoch_eval_losses = training_results_dict['epoch_eval_losses']
epoch_eval_metrics = training_results_dict['epoch_eval_metrics']
epoch = training_results_dict['epoch']
print(epoch_train_losses, epoch_eval_losses, epoch_eval_metrics, sep='\n')

  0%|          | 0/25000 [00:00<?, ?it/s]

  0%|          | 0/12500 [00:00<?, ?it/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
<ipython-input-3-8a6715338695>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_batch = torch.tensor(batch['labels'])


Epoch 0: Train Loss = 2.6839280128479004, Eval Loss = 2.0217185020446777
Save this model (y/n)?> n
Continue training (y/n)?> y
Epoch 1: Train Loss = 1.6994225978851318, Eval Loss = 1.772935390472412
Save this model (y/n)?> n
Continue training (y/n)?> y
Epoch 2: Train Loss = 1.5108166933059692, Eval Loss = 1.7279548645019531
Save this model (y/n)?> y
Continue training (y/n)?> y


<ipython-input-3-8a6715338695>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_batch = torch.tensor(batch['labels'])


...

Epoch 6: Train Loss = 0.7844387292861938, Eval Loss = 1.6836659908294678
Save this model (y/n)?> y
Continue training (y/n)?> y


<ipython-input-3-8a6715338695>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_batch = torch.tensor(batch['labels'])


Epoch 7: Train Loss = 0.7902386784553528, Eval Loss = 1.6700924634933472
Save this model (y/n)?> y
Continue training (y/n)?> y


<ipython-input-3-8a6715338695>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_batch = torch.tensor(batch['labels'])


Epoch 8: Train Loss = 0.7310791015625, Eval Loss = 1.6936670541763306
Save this model (y/n)?> n
Continue training (y/n)?> n
[2.6839280128479004, 1.6994225978851318, 1.5108166933059692, 1.352116584777832, 1.0249603986740112, 1.2612980604171753, 0.7844387292861938, 0.7902386784553528, 0.7310791015625]
[2.0217185020446777, 1.772935390472412, 1.7279548645019531, 1.7150275707244873, 1.708227515220642, 1.6990811824798584, 1.6836659908294678, 1.6700924634933472, 1.6936670541763306]
[]


In [ ]:
model.save_pretrained(f'lmqg_squad_reduced_mt5base_conj_10000examples_(epoch {epoch})')

### 5. Analysis of the Results

### Calculating BLEU and ROUGE score

In [ ]:
metrics = {
    'bleu': get_bleu_config(tokenizer),
    'nist_m': get_nist_config(tokenizer),
    'rouge': get_rouge_config(tokenizer),
}

In [ ]:
P = True

In [ ]:
if P:
  model = MT5ForConditionalGeneration.from_pretrained('CleanedUp/lmqg_squad_reduced_mt5base_conj_10000examples_test_epoch7', local_files_only=True)

In [ ]:
if P:
  optimizer, train_dataloader, eval_dataloader, lr_scheduler, \
    num_training_steps = get_training_configuration(train_batch_size=4, eval_batch_size=4, tokenizer=tokenizer, learning_rate=1e-3, num_epochs=2)

5000


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
if P:
  device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
  model.to(device)
  print()

In [ ]:
tokenized_test_dataset = tokenized_test_dataset.shuffle(seed=42)
test_dataset = test_dataset.shuffle(seed=42)

In [ ]:
from tqdm.auto import tqdm
test_dataloader = DataLoader(tokenized_test_dataset, shuffle=True, batch_size=8, collate_fn=DataCollatorWithPadding(tokenizer=tokenizer))
test_loss_tracker, test_metrics_tracker, num_test_steps = [], [], len(test_dataloader)
test_progress_bar = tqdm(range(num_test_steps))
test_loss = evaluation_loop(
    model, device, optimizer, test_dataloader, lr_scheduler, test_loss_tracker, test_metrics_tracker, metrics, test_progress_bar,
    tokenizer=tokenizer, num_beams=4, num_candidates=4, tokenize_predictions_output=False,
)

  0%|          | 0/1485 [00:00<?, ?it/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
<ipython-input-2-8a6715338695>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_batch = torch.tensor(batch['labels'])
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `None` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:377: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `None` -- this flag is only used in sa

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Metrics = {'bleu': {'bleu': 0.24426754103370749, 'precisions': [0.5720669527233369, 0.30484898014778494, 0.19655084240134102, 0.13184796295110657], 'brevity_penalty': 0.9420973353412331, 'length_ratio': 0.9437107835984241, 'translation_length': 129345, 'reference_length': 137060}, 'nist_m': {'nist_mt': 6.419595998586757}, 'rouge': {'rouge1': 0.5600562362586241, 'rouge2': 0.32816017305282763, 'rougeL': 0.5208039316900042, 'rougeLsum': 0.5210896648639034}}


In [ ]:
test_loss_tracker

[1.5450189113616943]

In [ ]:
test_dataset[1008:1020]['question']

['What was the design name of the Chinese Olympic Torch?',
 'When did Beyoncé receive the Legend Award?',
 'What was the magnitude of the southern Sichuan quake?',
 'By how much did opening these jobs to women increase the recruiting pool by?',
 'About how many Montanans died in the war?',
 'What kind of energy consists of the light and heat provided by the Sun?',
 'What kind of classes are not offered in Mittelschule?',
 'What movie did Beyonce act in 2006?',
 'Who attacked Scout and Jem?',
 'Hinduism and Christianity use what term for a permanent self?',
 "What professors established the importance of Whitehead's work?",
 'How much did advertisers spend for a 30 second ad spot during American Idol in 2014?']

In [ ]:
final_predictions = select_best_output(
    model, tokenizer, tokenized_test_dataset['input_ids'][1008:1020].to(device), test_dataset[1008:1020]['question'], score_function=bertscore_f1based_score,
    max_length=64, num_beams=4, top_k=None, top_p=None, num_candidates=4, verbose=False, tokenize_output=False
)

In [ ]:
final_predictions

['What is the design of the Olympic Torch?',
 'When did Beyoncé receive the Legend Award?',
 'What earthquake caused the earthquake on August 30, 2008 in southern Sichuan?',
 'How much of the potential recruiting pool did the introduction of women into the combat arms increased the potential recruiting pool?',
 'How many Montanans died in the war?',
 'What is radiant light and heat from the Sun harnessed?',
 'What kind of classes does the Mittelschule offer?',
 'What was the name of the film that Queen starred opposite Steve Martin in 2006?',
 'Who is humiliated by the events of the trial?',
 'What is a permanent self called in Hinduism and Christianity?',
 "What were the names of some professors that influenced Whitehead's work?",
 'How much did a 30-second spot cost for American Idol in 2014?']

In [ ]:
bleu = load('bleu')

In [ ]:
example_labels = tokenized_test_dataset['labels'][1008:1020]
example_labels[example_labels == -100] = 0

In [ ]:
tokenizer.batch_decode(example_labels, skip_special_tokens=True)

['What was the design name of the Chinese Olympic Torch?',
 'When did Beyoncé receive the Legend Award?',
 'What was the magnitude of the southern Sichuan quake?',
 'By how much did opening these jobs to women increase the recruiting pool by?',
 'About how many Montanans died in the war?',
 'What kind of energy consists of the light and heat provided by the Sun?',
 'What kind of classes are not offered in Mittelschule?',
 'What movie did Beyonce act in 2006?',
 'Who attacked Scout and Jem?',
 'Hinduism and Christianity use what term for a permanent self?',
 "What professors established the importance of Whitehead's work?",
 'How much did advertisers spend for a 30 second ad spot during American Idol in 2014?']

In [ ]:
bleu.compute(predictions=final_predictions, references=test_dataset['question'][1008:1020])

{'bleu': 0.24350374517283724,
 'precisions': [0.5586206896551724,
  0.3007518796992481,
  0.19008264462809918,
  0.11009174311926606],
 'brevity_penalty': 1.0,
 'length_ratio': 1.124031007751938,
 'translation_length': 145,
 'reference_length': 129}

In [ ]:
bert_score = load('bertscore')

In [ ]:
bert_score.compute(predictions=final_predictions, references=test_dataset['question'][1008:1020], lang='en')

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'precision': [0.977583646774292,
  1.0,
  0.9233590364456177,
  0.9060705304145813,
  0.9854481220245361,
  0.9166568517684937,
  0.96336430311203,
  0.8915995359420776,
  0.8799552917480469,
  0.9034591317176819,
  0.9107999801635742,
  0.9610221982002258],
 'recall': [0.9570581912994385,
  1.0,
  0.9365372657775879,
  0.9185553789138794,
  0.9783714413642883,
  0.9007253646850586,
  0.9531168341636658,
  0.9106607437133789,
  0.8809152841567993,
  0.9056779742240906,
  0.9455307722091675,
  0.9474698901176453],
 'f1': [0.9672120809555054,
  1.0,
  0.9299014806747437,
  0.9122702479362488,
  0.9818970561027527,
  0.9086212515830994,
  0.958213210105896,
  0.9010293483734131,
  0.880435049533844,
  0.9045671820640564,
  0.9278404712677002,
  0.954197883605957],
 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.32.1)'}

In [ ]:
test_dataset[990:1000]['question']

['When was Northern Rock taken into public hands?',
 'How many people work in the New York publishing industry?',
 'Until when did the Portuguese government resist decolonization of their overseas territories?',
 'Who gave Chopin a loan in September for an apartment?',
 'What was the U.S. unemployment rate in October 2009?',
 "Which team did Notre Dame's football team find inspiration from?",
 'What countries used comprehensive schools extensively?',
 'Who was the mayor of Nagano?',
 'Who did Ü-Tsang king have an alliance with?',
 'At what age did Kanye West relocate to China?']

In [ ]:
model.eval()
with torch.no_grad():
  predictions = model.generate(tokenized_test_dataset['input_ids'][990:1000].to(device), max_length=64, num_beams=10)

In [ ]:
tokenizer.batch_decode(predictions, skip_special_tokens=True)

['When was Northern Rock taken into public?',
 'How many people does the publishing industry employ?',
 'When did the CARNATION Revolution begin?',
 "Who supported Chopin's loan in September?",
 'What was the U.S. unemployment rate in 2009?',
 'Which team brought football to Notre Dame in 1887?',
 'What countries are comprehensive schools commonly used in relation to?',
 'Who was the Mayor of Nagano?',
 'Who did the U-Tsang king allied with?',
 'At what age did West move with his mother to Nanjing?']

In [ ]:
bleu.compute(predictions=tokenizer.batch_decode(predictions, skip_special_tokens=True), references=test_dataset['question'][990:1000])

{'bleu': 0.316333449674997,
 'precisions': [0.7032967032967034,
  0.41975308641975306,
  0.2676056338028169,
  0.19672131147540983],
 'brevity_penalty': 0.8959325840172705,
 'length_ratio': 0.900990099009901,
 'translation_length': 91,
 'reference_length': 101}

In [ ]:
bert_score.compute(predictions=tokenizer.batch_decode(predictions, skip_special_tokens=True), references=test_dataset['question'][990:1000], lang='en')

{'precision': [0.9634586572647095,
  0.9625447392463684,
  0.8753376603126526,
  0.9523909687995911,
  0.9958045482635498,
  0.9038297533988953,
  0.9044301509857178,
  0.997177004814148,
  0.9511581659317017,
  0.9357975721359253],
 'recall': [0.9477594494819641,
  0.9419648051261902,
  0.8841994404792786,
  0.9254592061042786,
  0.9863343238830566,
  0.9037783145904541,
  0.9512970447540283,
  0.997177004814148,
  0.9395004510879517,
  0.9566055536270142],
 'f1': [0.9555445909500122,
  0.9521435499191284,
  0.8797461986541748,
  0.9387319684028625,
  0.9910468459129333,
  0.9038040041923523,
  0.9272717833518982,
  0.997177004814148,
  0.9452933669090271,
  0.9460871815681458],
 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.32.1)'}

In [ ]:
bertscore = compute_bert_score(
    test_dataset, tokenized_test_dataset, model, device, tokenizer, batch_size=16,
    lang='en', model_type=None, max_length=200, num_beams=4, num_candidates=1
)

  0%|          | 0/743 [00:00<?, ?it/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
bertscore

{'precision': (0.922971893941249, 0.009505535089235642),
 'recall': (0.9198167614017088, 0.009513250055126524),
 'f1': (0.9212104170702268, 0.008915368096203947)}

In [ ]:
np.mean([v[0] for v in bertscore.values()])

0.9213330241377281